In [1]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.errors as errors
import azure.cosmos.http_constants as http_constants
import os
import pprint
import xlsxwriter
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill,Font,colors,Border,Side,Fill
import datetime
from time import strftime,gmtime,mktime,strptime
import config

In [2]:
url = 'https://alexformdb.documents.azure.com:443/'
key = 'fsRn9LNJU3OMggzUy9DuK5wYmCCxcYe234IZhEMNxwMiUHVw1JDxHmC5kJg7QAZc7XROhQcWd5VurrNf3dqawg=='
client = cosmos_client.CosmosClient(url, {'masterKey': key})
DATABASE_ID = 'AlexGraphApi'
COLLECTION_ID = 'v1'
database_link = 'dbs/' + DATABASE_ID
collection_link = database_link + '/colls/' + COLLECTION_ID
documentlist = list(client.ReadItems(collection_link, {'maxItemCount':100}))

#print(documentlist)
print('Found {0} documents'.format(documentlist.__len__()))

Found 99438 documents


In [3]:
def getList(dict): 
    newdict = []
    for i in dict.keys():
        newdict.append(i)
    return newdict
def allID():
    return config.getIDs()
def nameList():
    newdict = []
    temp = config.getNames()
    for i in  config.getNames():
        newdict.append(temp[i])
    return newdict
def getDate(Datestr):
    newDate = strftime("%Y-%m-%d", gmtime(mktime(strptime(Datestr, "%a %b  %d %H:%M:%S %Y"))))
    return newDate
def getGroupName():
    group = allID()
    name = getList(group)
    return name
def getTimeing(Timestr):
    newTime = strftime("%H:%M", gmtime(mktime(strptime(Timestr, "%a %b  %d %H:%M:%S %Y"))))
    b = newTime
    s = int(b[0:2]) + 8
    if(s < 10):
        b = b.replace(b[0:2],"0" + str(s))
    elif(s > 24):
        s = s - 24
        b = b.replace(b[0:2],"0" + str(s))
    else:
        b = b.replace(b[0:2],str(s))
    if(b[4] == "4"):
        b = b[:4] + "5"
    if(b[4] == "9"):
        t1 = int(b[3:5]) + 1
        if(t1 == 60):
            s = int(b[0:2]) + 1
            if(s < 10):
                b = b.replace(b[0:2],"0" + str(s))
            elif(s > 24):
                s = s - 24
                b = b.replace(b[0:2],"0" + str(s))
            else:
                b = b.replace(b[0:2],str(s))
            b = b[:3] + "00"
        else:
            b = b[:3] + str(t1)
    if(b[4] == "8"):
        b = b[:3] + "00"
    return b


In [4]:
def groupClean():
    groupWash = []
    for i in documentlist:   
        #pprint.pprint(i)
        index = {}
        if(i.get('groupName') is not None):
            index['groupName'] = i['groupName']
            index['records'] = i['records']
            index['timestamp'] = i['timestamp']
            groupWash.append(index)
    return groupWash

In [5]:
groupName = getGroupName()
print(groupName)

['NT0300', 'NT0P00', 'NT0100', 'NT0M00', 'NT0000', 'NT0C00', 'NT0900', 'NT0220']


In [6]:
#pprint.pprint(groupClean())

In [7]:
def getOutform():
    outform = []
    groupList = groupClean()
    name = nameList()
    for i in name:
        temp = {}
        temp['displayname'] = i
        outform.append(temp)

    #pprint.pprint(outform)

    for i in groupList:
        for j in i['records']:
            for n in outform:
                if(j['displayname'] == n['displayname']):
                    n['groupName'] = i['groupName']
                    n[i['timestamp']] = j['activity']
    return outform

In [8]:
def getDateList():    
    groupP = getOutform()
    timeA = groupP[0]
    del timeA['displayname']
    del timeA['groupName']
    Datelist = []
    for i in timeA:
        tempA = getDate(i)
        boolA = False
        for j in Datelist:
             if(j == tempA):
                boolA = True
        if(boolA == False):
            Datelist.append(tempA)
    return Datelist

In [9]:
def getData(listing):
    groupP = getOutform()
    Datelist = getDateList()
    tempDict = []
    Ctemp = groupP.copy()
    for i in Ctemp:
        tempp = getList(i)
        tempp.remove("displayname")
        tempp.remove("groupName")
        for j in tempp:
            #print(j)
            if(getDate(j) != Datelist[listing]):
                 del i[j]
    a = Ctemp
    DictT = {}
    formT = []
    for i in a:
        DictT = {}
        for j in i:
            if(j != 'displayname' and j !='groupName'):
                DictT[getTimeing(j)] = i[j]
            else:
                DictT[j] = i[j]
        formT.append(DictT)
    
    return(formT)


In [10]:
def toColorful(filename):
    preColor = load_workbook(filename)
    sheetname = getDateList()
    
    for c in sheetname:
        sheet = preColor[c]
        RED = PatternFill(start_color = "FF0000",
                         end_color = "FF0000",
                         fill_type = "solid")
        GREEN = PatternFill(start_color = "00FF00",
                         end_color = "00FF00",
                         fill_type = "solid")
        YELLOW = PatternFill(start_color = "FFBB00",
                         end_color = "FFBB00",
                         fill_type = "solid")
        GRAY = PatternFill(start_color = "888888",
                         end_color = "888888",
                         fill_type = "solid")
        BLACK = PatternFill(start_color = "000000",
                         end_color = "000000",
                         fill_type = "solid")
        ORANGE = PatternFill(start_color = "FFAA33",
                         end_color = "FFAA33",
                         fill_type = "solid")
        LYELLOW = PatternFill(start_color = "FFFF33",
                         end_color = "FFFF33",
                         fill_type = "solid")
        
        
        T1= PatternFill(start_color = "5B00AE",
                         end_color = "5B00AE",
                         fill_type = "solid")
        T2 = PatternFill(start_color = "8600FF",
                         end_color = "8600FF",
                         fill_type = "solid")
        T3 = PatternFill(start_color = "9F35FF",
                         end_color = "9F35FF",
                         fill_type = "solid")
        T4 = PatternFill(start_color = "CA8EFF",
                         end_color = "CA8EFF",
                         fill_type = "solid")
        T5 = PatternFill(start_color = "DCB5FF",
                         end_color = "DCB5FF",
                         fill_type = "solid")
        T6 = PatternFill(start_color = "B766AD",
                         end_color = "B766AD",
                         fill_type = "solid")
        T7 = PatternFill(start_color = "DAB1D5",
                         end_color = "DAB1D5",
                         fill_type = "solid")
        T8 = PatternFill(start_color = "9999CC",
                         end_color = "9999CC",
                         fill_type = "solid")
        
        
        
        

        for i in sheet:
            for j in i:
                if(j.value == "Available"):
                    j.fill = GREEN
                elif(j.value == "Away"):
                    j.fill = ORANGE
                elif(j.value == "BeRightBack"):
                    j.fill = YELLOW
                elif(j.value == "Busy"):
                    j.fill = RED
                elif(j.value == "DoNotDisturb"):
                    j.fill = RED
                elif(j.value == "InACall"):
                    j.fill = RED
                elif(j.value == "InAConferenceCall"):
                    j.fill = YELLOW
                elif(j.value == "Inactive"):
                    j.fill = YELLOW
                elif(j.value == "InAMeeting"):
                    j.fill = LYELLOW
                elif(j.value == "Offline"):
                    j.fill = GRAY
                elif(j.value == "OffWork"):
                    j.fill = BLACK
                elif(j.value == "OutOfOffice"):
                    j.fill = YELLOW
                elif(j.value == "PresenceUnknown"):
                    j.fill = BLACK
                elif(j.value == "UrgentInterruptionsOnly"):
                    j.fill = YELLOW
                elif(j.value == "Presenting"):
                    j.fill = YELLOW
                elif(j.value == "InAConferenceCall"):
                    j.fill = YELLOW
                elif(j.value == "NT0220"):
                    j.fill = T1
                elif(j.value == "NT0900"):
                    j.fill = T2
                elif(j.value == "NT0C00"):
                    j.fill = T3
                elif(j.value == "NT0000"):
                    j.fill = T4
                elif(j.value == "NT0M00"):
                    j.fill = T5
                elif(j.value == "NT0100"):
                    j.fill = T6
                elif(j.value == "NT0P00"):
                    j.fill = T7
                elif(j.value == "NT0300"):
                    j.fill = T8
    preColor.save(filename)

In [11]:
def toExcel():
    timeformat = '%Y%m%d%H%M'
    date = datetime.datetime.now().strftime(timeformat)
    filename = "/Users/alex.c.lo/Desktop/WiadvanceWork/006-WiAdvanceGetPre/userPresence/"+str(date)+'.xlsx'
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    tt = getDateList()
    for i in range(len(tt)):
        formOut = getData(i)
        
        formxsl1 = pd.DataFrame(formOut)
        formxsl1.to_excel(writer,tt[i], index=False)
    writer.save()
    toColorful(filename)
    print("done")

In [12]:
toExcel()

done
